In [1]:
# Importe de librerias que se van a utilizar
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Activation
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model
from tensorflow.keras.callbacks import TensorBoard
import time

# Configuracion inicial de la red neuronal
img_width, img_height = 70,70 # width y height para las imagen de entrada
input_depth = 1 #1: Transformación a escala de grises
train_data_dir = 'D:/Datasets/testchairs/training' # Direccion de la carpeta de training
testing_data_dir = 'D:/Datasets/testchairs/testing' # Direccion de la carpeta de testing
epochs = 10 # Numero de epochs a utilizar
batch_size = 5 # Numero de Batches a utilizar

# Definicion de la generacion de imagenes para Keras,
# Nomalizacion de las imagenes de entrada, (valores de 0 a 1)
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

# Lectura de las imagenes por batch
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode='grayscale',
    target_size=(img_width,img_height),# Tamano de las imagenes de entrada
    batch_size=batch_size,# Tamaño batch
    shuffle=True,# aleatorizacion de los datos
    class_mode='categorical')# Definición del modelo de red neuronal como categorico
testing_generator = test_datagen.flow_from_directory(
    testing_data_dir,
    color_mode='grayscale',
    target_size=(img_width,img_height),
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical')

# Confirmacion de la configuracion de la red neuronal
print(train_generator.class_indices)
print(testing_generator.class_indices)

#define input image order shape
if K.image_data_format() == 'channels_first':
    input_shape_val = (input_depth, img_width, img_height)
else:
    input_shape_val = (img_width, img_height, input_depth)
print(input_shape_val)

dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128, 256]
conv_layers = [1, 2, 3]

for desnse_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, desnse_layer, int(time.time()))
            print(NAME)
            #define the network
            model = Sequential()
            # Layer 1
            model.add(Conv2D(layer_size, (5, 5), 
                            input_shape=input_shape_val, 
                            padding='same', name='input_tensor'))
            model.add(Activation('relu'))
            model.add(MaxPool2D((2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (5, 5), padding='same'))
                model.add(Activation('relu'))
                model.add(MaxPool2D((2, 2)))

            # flattening the model for fully connected layer
            model.add(Flatten())
            for l in range(desnse_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            # output layer
            model.add(Dense(train_generator.num_classes, 
                            activation='softmax', name='output_tensor'))

            tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

            # Compilile the network
            model.compile(loss='categorical_crossentropy',
                        optimizer='sgd', metrics=['accuracy'])
            # Show the model summary
            #model.summary()
            # Train and test the network
            model.fit(
                train_generator,#our training generator
                #number of iteration per epoch = number of data / batch size
                steps_per_epoch=np.floor(train_generator.n/batch_size),
                epochs=epochs,#number of epoch
                validation_data=testing_generator,#our validation generator
                #number of iteration per epoch = number of data / batch size
                validation_steps=np.floor(testing_generator.n / batch_size),
                callbacks=[tensorboard])

print("Training is done!")
model.save('./model/{}.h5'.format(NAME))
print("Model is successfully stored!")

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'D:/Datasets/testchairs/training'